# Análise de Emoções em tweets relacionados à pandemia de Covid-19

## Introdução

* Motivação
* 

In [1]:
import pandas as pd
import numpy as np
import spacy
import os
import pickle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from spacy.cli.download import download

In [2]:
def criar_embeddings(df_tweets):
    try:
        nlp = spacy.load('pt_core_news_lg')
    except (IOError, OSError):
        download('pt_core_news_lg')
        nlp = spacy.load('pt_core_news_lg')
    # desativamos todos os outros pipes que vem com o modelo nlp porque não preicsaremos deles
    with nlp.disable_pipes():
        # transformamos cada texto em um vetor e colocamos em uma array
        print('Fazendo os word embeddings')
        vetores = np.array([nlp(texto).vector for texto in df_tweets.Texto])

    return vetores


def ler_modelo(path: str):
    return pickle.load(open(path, 'rb'))


def salvar_modelo(path: str, modelo):
    return pickle.dump(modelo, open(path, 'wb'))    


def fazer_amostragem(train_dataset: pd.DataFrame):
    """

    :param train_dataset:
    :return:
    """
    sentimentos = train_dataset['Sentimento'].unique()
    df = pd.DataFrame([])

    for sentimento in sentimentos:
        df_filtrado = train_dataset.loc[train_dataset['Sentimento'] == sentimento][:600]
        df = pd.concat([df, df_filtrado])

    return df

In [3]:
path_datasets = '../resources/datasets'

# importação dos dados
df_treinamento = pd.read_csv(
    f'{path_datasets}/tweets_ekman.csv',
    usecols=['Texto', 'Sentimento']
).dropna()
df_alvo = pd.read_csv(f'{path_datasets}/tweets_pandemia.csv')

x_train, x_test, y_train, y_test = train_test_split(
    df_treinamento,
    df_treinamento['Sentimento'],
    test_size=0.2,
    random_state=42
)

In [4]:
# processamento dos dados para word embeddings
path_embeddings_treinamento = '../resources/modelos/embeddings_treinamento.pkl'
path_embeddings_teste = '../resources/modelos/embeddings_teste.pkl'

if os.path.exists(path_embeddings_treinamento):
    embeddings = ler_modelo(path_embeddings_treinamento)
else:
    embeddings = criar_embeddings(x_train)
    salvar_modelo(path_embeddings_treinamento, embeddings)

if os.path.exists(path_embeddings_teste):
    embeddings_teste = ler_modelo(path_embeddings_teste)
else:
    embeddings_teste = criar_embeddings(x_test)
    salvar_modelo(path_embeddings_teste, embeddings_teste)

In [5]:
path_svc = '../resources/modelos/svc.pkl'
path_lgr = '../resources/modelos/logistic_regression.pkl'
path_forest = '../resources/modelos/forest.pkl'

if os.path.exists(path_svc):
    svc = ler_modelo(path_svc)
else:
    svc = LinearSVC(C=100, random_state=0, dual=True, max_iter=10000)
    svc.fit(embeddings, y_train)
    salvar_modelo(path_svc, svc)
    
if os.path.exists(path_lgr):
    lgr = ler_modelo(path_lgr)
else:
    lgr = LogisticRegression(random_state=0)
    lgr.fit(embeddings, y_train)
    salvar_modelo(path_lgr, lgr)

if os.path.exists(path_forest):
    forest = ler_modelo(path_forest)
else:
    forest = RandomForestClassifier(random_state=0, n_jobs=-1)
    forest.fit(embeddings, y_train)
    salvar_modelo(path_forest, forest)

In [6]:
# previsões
previsoes_svc = svc.predict(embeddings_teste)
previsoes_lgr = lgr.predict(embeddings_teste)
previsoes_forest = forest.predict(embeddings_teste)

In [7]:
# testar performance
print('Relatório Linear SVC')
print(classification_report(y_test, previsoes_svc))

Relatório Linear SVC
              precision    recall  f1-score   support

      Neutro       0.96      0.94      0.95       495
       feliz       0.74      0.45      0.56      1931
        medo       0.35      0.83      0.50      2320
        nojo       0.50      0.24      0.32      1648
       raiva       0.13      0.18      0.16       562
      triste       0.45      0.08      0.14      2097

    accuracy                           0.43      9053
   macro avg       0.52      0.45      0.44      9053
weighted avg       0.50      0.43      0.40      9053


In [8]:
print('Relatório Logistic Regression')
print(classification_report(y_test, previsoes_lgr))

Relatório Logistic Regression
              precision    recall  f1-score   support

      Neutro       0.97      0.94      0.96       495
       feliz       0.71      0.73      0.72      1931
        medo       0.54      0.61      0.57      2320
        nojo       0.61      0.63      0.62      1648
       raiva       0.45      0.12      0.19       562
      triste       0.52      0.53      0.53      2097

    accuracy                           0.61      9053
   macro avg       0.63      0.59      0.60      9053
weighted avg       0.60      0.61      0.60      9053


In [9]:
print('Relatório Random Forest')
print(classification_report(y_test, previsoes_forest))

Relatório Random Forest
              precision    recall  f1-score   support

      Neutro       0.95      0.87      0.91       495
       feliz       0.63      0.66      0.64      1931
        medo       0.45      0.56      0.50      2320
        nojo       0.48      0.43      0.46      1648
       raiva       0.96      0.05      0.09       562
      triste       0.45      0.47      0.46      2097

    accuracy                           0.52      9053
   macro avg       0.66      0.51      0.51      9053
weighted avg       0.55      0.52      0.51      9053


In [10]:
pontuacoes_logreg = cross_val_score(lgr, embeddings_teste, y_test, cv=5, n_jobs=-1, scoring='f1_weighted')
pontuacoes_svc = cross_val_score(svc, embeddings_teste, y_test, cv=5, n_jobs=-1, scoring='f1_weighted')
pontuacoes_forest = cross_val_score(forest, embeddings_teste, y_test, cv=5, n_jobs=-1, scoring='f1_weighted');

In [11]:
def exibir_pontuacoes(pontuacoes):
    soma_ponuacoes = 0

    for valor in pontuacoes:
        soma_ponuacoes += valor
        
    media = soma_ponuacoes / len(pontuacoes)
    
    print(f'Lista de pontuações: {pontuacoes}\nMédia: {media}' )
    
exibir_pontuacoes(pontuacoes_logreg)
exibir_pontuacoes(pontuacoes_svc)
exibir_pontuacoes(pontuacoes_forest)

Lista de pontuações: [0.56582297 0.58788242 0.574865   0.5794129  0.56741404]
Média: 0.5750794670048374
Lista de pontuações: [0.44221795 0.51446641 0.35366093 0.47581766 0.47062506]
Média: 0.45135760052701085
Lista de pontuações: [0.47940858 0.49138109 0.46536405 0.46713247 0.48691761]
Média: 0.4780407583044698


O modelo Logistic Regression obteve o melhor desempenho dentre os modelos, enquanto que o LinearSVC e o Random Forest tiveram desempenhos semelhantes.

## Trabalhos Futuros
Realizar o ajuste de hiperparâmetros para tentar obter um modelo de logistic regression com um desempenho ainda superior.